# Trabalho Prático - Modulo 2 - BootCamp Data Science - IGTI
Trabalho desenvolvida com a implenetação do uso do FrameWork PySpark com Apach Spark


In [1]:
from pyspark.sql import SparkSession

# Inicio da seção do Spark
spark = SparkSession \
    .builder \
    .appName("Trabalho Pratico  - Cientista de Dados - Apache Spark") \
    .getOrCreate()

spark.version


'3.3.0'

In [2]:
# Carregando a planilha CSV que contém o dataset
sp500_df = spark.read.csv('C:/Users/erico/Documents/103 Data Science/IGTI - BootCamps/Bootcamp Data Science/Módulo 2/all_stocks_5yr.csv',header='True',inferSchema='True')

# Detalhes dos atributos 
sp500_df.printSchema()


root
 |-- date: timestamp (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: integer (nullable = true)
 |-- Name: string (nullable = true)



### Pergunt 1
#### Quantos registros há na planilha?

In [3]:
sp500_df.count()

619040

In [4]:
sp500_df.show(5)

+-------------------+-----+-----+-----+-----+--------+----+
|               date| open| high|  low|close|  volume|Name|
+-------------------+-----+-----+-----+-----+--------+----+
|2013-02-08 00:00:00|15.07|15.12|14.63|14.75| 8407500| AAL|
|2013-02-11 00:00:00|14.89|15.01|14.26|14.46| 8882000| AAL|
|2013-02-12 00:00:00|14.45|14.51| 14.1|14.27| 8126000| AAL|
|2013-02-13 00:00:00| 14.3|14.94|14.25|14.66|10259500| AAL|
|2013-02-14 00:00:00|14.94|14.96|13.16|13.99|31879900| AAL|
+-------------------+-----+-----+-----+-----+--------+----+
only showing top 5 rows



In [5]:
sp500_df.describe('close', 'volume').show()

+-------+-----------------+-----------------+
|summary|            close|           volume|
+-------+-----------------+-----------------+
|  count|           619040|           619040|
|   mean|83.04376276476519|4321823.395568945|
| stddev|97.38974800165782|8693609.511967566|
|    min|             1.59|                0|
|    max|           2049.0|        618237630|
+-------+-----------------+-----------------+



### Pergunta 2
#### Quantos registros há na planilha para a ação da Apple (AAPL)?

In [6]:
from pyspark.sql.functions import col

sp500_df.filter(col('Name') == 'AAPL').count()

1259

### Pergunta 3
#### Quantas empresas distintas têm registros nessa planilha?

In [7]:
from pyspark.sql.functions import countDistinct

sp500_df.select(countDistinct('Name')).show()

+--------------------+
|count(DISTINCT Name)|
+--------------------+
|                 505|
+--------------------+



### Pergunta 4
#### Com qual frequência o preço de uma ação no fechamento é maior do que o preço na abertura?

In [97]:
a = sp500_df.filter(sp500_df.close > sp500_df.open).count()
b = sp500_df.count()

a/b*100

51.52655724993538

### Pergunta 5
#### Qual o maior valor das ações da Apple (AAPL) na história?

In [104]:
sp500_df.filter(sp500_df.Name == 'AAPL').describe('high').show()


+-------+------------------+
|summary|              high|
+-------+------------------+
|  count|              1259|
|   mean|109.95111834789516|
| stddev|30.686186407983914|
|    min|           57.0857|
|    max|             180.1|
+-------+------------------+



### Pergunta 6
#### Qual ação tem a maior volatilidade? Uma forma é medir o desvio-padrão do preço de fechamento de cada ação e considerar a ação de maior desvio-padrão.

In [10]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql import Window as W


#Usando uma Window Function para particionar o DataFrame pelo código da ação:
w = W.partitionBy('Name')

#Adicionando uma coluna com o desvio padrão calculado na pela partição de nome de ação:
sp500_df = sp500_df.withColumn('std', F.stddev('close').over(w))

sp500_df\
    .select('Name','std')\
    .orderBy(F.desc('std'))\
    .limit(1)\
    .show()



+----+----------------+
|Name|             std|
+----+----------------+
|PCLN|320.533473018748|
+----+----------------+



### Pergunta 7
#### Qual o dia com maior volume total de negociação da bolsa?

In [24]:
df = sp500_df\
    .select('date','volume')\
    .groupBy('date')\
    .sum('volume')

df.orderBy(col('sum(volume)').desc()).show(1)

+-------------------+-----------+
|               date|sum(volume)|
+-------------------+-----------+
|2015-08-24 00:00:00| 4607945196|
+-------------------+-----------+
only showing top 1 row



### Pergunta 8
#### Qual a ação mais negociada da bolsa, em volume de transações?

In [23]:
df1 = sp500_df\
    .select('Name','volume')\
    .groupBy('Name')\
    .sum('volume')

df1.orderBy(col('sum(volume)').desc()).show(1)

+----+------------+
|Name| sum(volume)|
+----+------------+
| BAC|117884953591|
+----+------------+
only showing top 1 row



### Pergunta 9
#### Quantas ações começam com a letra “A”?

In [93]:
df2 = sp500_df.dropDuplicates(['Name'])

df2.filter(df2.Name.startswith('A')).count()

59

### Pergunta 10
#### Com qual frequência o preço mais alto do dia da ação também é o preço de fechamento?

In [31]:
a = sp500_df.filter(sp500_df.close == sp500_df.high).count()
b = sp500_df.count()

a/b*100

1.1986301369863013

### Pergunta 11
#### Em qual dia a ação da Apple mais subiu entre a abertura e o fechamento, de forma absoluta?

In [46]:
df3 = sp500_df.filter((sp500_df.Name == 'AAPL') &\
                (sp500_df.open < sp500_df.close))

df3 = df3.withColumn('dif', col('close') - col('open'))

df3.select('date','dif').orderBy(col('dif').desc()).show(1)

+-------------------+----+
|               date| dif|
+-------------------+----+
|2015-08-24 00:00:00|8.25|
+-------------------+----+
only showing top 1 row



### Pergunta 12
#### Em média, qual o volume diário de transações das ações da AAPL?

In [98]:
media = sp500_df.groupBy('Name')\
        .agg(F.mean('volume').cast('int').alias('media'))\
        .filter(col('Name') == 'AAPL')\
        .show()

+----+--------+
|Name|   media|
+----+--------+
|AAPL|54047899|
+----+--------+



### Pergunta 13
#### Quantas ações tem 1, 2, 3, 4 e 5 caracteres em seu nome, respectivamente?

In [86]:
df4 = sp500_df.withColumn('sizeName', F.length('name'))
df4 = df4.dropDuplicates(['Name'])
lista = []
for x in [1,2,3,4,5]:
    lista.append(df4.filter(col('sizeName') == x).count())
    
print(lista)   

[10, 50, 323, 117, 5]


### Pergunta 14
#### Qual a ação menos negociada da bolsa, em volume de transações?

In [87]:
df1 = sp500_df\
    .select('Name','volume')\
    .groupBy('Name')\
    .sum('volume')

df1.orderBy(col('sum(volume)').asc()).show(1)

+----+-----------+
|Name|sum(volume)|
+----+-----------+
|APTV|   92947779|
+----+-----------+
only showing top 1 row



### Pergunta 15
#### Qual dessas afirmações NÃO ilustra uma diferença entre as abstrações RDD e Dataframe?

  - **O Dataframe não é tolerante a falhas, enquanto o RDD sim (O “R” é de “resiliente”).**
  - O Dataframe suporta diretamente manipulação de colunas. 
  - O RDD tem uma API mais flexível e livre que o Dataframe, dado que ele trabalha com objetos arbitrários. 
  - Computações com Dataframes podem ser mais eficientes que com RDDs, pelo fato do Spark conhecer as colunas sobre as quais vai operar.